#mounting of directory used in Boris' implementation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ACG Lab/Dyakov et al 2023 (google drive version)/Cluster analysis/7013 MSPLIT')

Mounted at /content/drive


#install all dependencies

In [ ]:
!pip install gprofiler-official

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as PathEffects
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.decomposition import NMF, PCA
from sklearn.manifold import TSNE
from gprofiler import GProfiler
from collections import Counter
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from scipy.cluster.hierarchy import dendrogram, linkage
import csv
import pickle
from scipy.optimize import linear_sum_assignment
from sklearn.mixture import GaussianMixture
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')

#NMF analysis

Using the SAINT results for the BirA* dataset, this code runs the NMF analysis which produces the files included in Supplementary Table 5:


*   Prey matrix
*   Bait matrix
*   Clusters (gene lists)
*   Gene set enrichment analysis results using g:Profiler

These results are displayed in Figs. 2 and 3 and Extended Data Figs. 5-7




In [ ]:
def nmf_cluster_analysis_filtering(n_components):
    def norm_avg_ctrl(df):
        ctrls = [i.split('|') for i in df['ctrlCounts']]
        ### choose either of the next two lines depending on if you wanna use all controls for subtraction ###
        ### or top 10 control counts per prey (effectively control compression = 10) ###
        #sums = [sum([int(element) for element in ctrl])/len(ctrls[0]) for ctrl in ctrls] #allctrls
        sums = [sum(sorted([int(element) for element in ctrl], reverse=True)[:10])/10 for ctrl in ctrls] #top 10 only
        df['AvgCtrl'] = sums
        df['CorrectedAvgSpec'] = df['AvgSpec'] - df['AvgCtrl']
        df = df[df['BFDR'] <= 0.01]
        df = df.pivot_table(index=['Bait'], columns=['PreyGene'], values=['CorrectedAvgSpec'])
        df = df.fillna(0)
        df = df.clip(lower=0)
        df.columns = df.columns.droplevel()
        scaler = MinMaxScaler()
        df_norm = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)
        return df_norm

    # Read and preprocess the data
    df_original = pd.read_csv('7013_cleaned_v2.txt', sep='\t')
    df_original = norm_avg_ctrl(df_original)

    # Perform NMF clustering
    nmf = NMF(n_components=n_components, init='nndsvd', l1_ratio=1, random_state=46)
    scores_original = nmf.fit_transform(df_original)
    basis_original = nmf.components_.T

    scores_df_original = pd.DataFrame(scores_original)
    basis_df_original = pd.DataFrame(basis_original)

    # Set the index to prey names
    basis_df_original.index = df_original.columns
    scores_df_original.index = df_original.index

    # Find the maximum score for each component in the basis matrix
    max_scores_per_column = basis_df_original.max()

    # Apply the XX% threshold check for primary, secondary, and tertiary scores
    primary_cluster = basis_df_original.idxmax(axis=1)
    secondary_cluster = basis_df_original.apply(lambda row: row.nlargest(2).idxmin(), axis=1)
    tertiary_cluster = basis_df_original.apply(lambda row: row.nlargest(3).idxmin(), axis=1)

    # Update the basis_matrix_original with new columns
    basis_matrix_original = basis_df_original.copy()

    # Calculate primary, secondary, and tertiary scores
    for idx, row in basis_df_original.iterrows():
        sorted_scores = row.sort_values(ascending=False)

        # Primary score and cluster
        primary_score = sorted_scores.iloc[0]
        primary_cluster_id = primary_cluster[idx]
        basis_matrix_original.at[idx, 'primary score'] = primary_score
        # Check if primary cluster meets the thresholds
        if primary_score >= 0.05 * max_scores_per_column[primary_cluster_id]:
            basis_matrix_original.at[idx, 'primary cluster'] = primary_cluster_id
        else:
            basis_matrix_original.at[idx, 'primary cluster'] = 'NA'

        # Secondary score and cluster
        secondary_cluster_id = secondary_cluster[idx]
        if len(sorted_scores) > 1 and secondary_cluster_id != 'NA':
            secondary_score = sorted_scores.iloc[1]
            # Check if secondary cluster meets the thresholds
            if secondary_score >= 0.70 * primary_score and secondary_score >= 0.05 * max_scores_per_column[secondary_cluster_id]:
                basis_matrix_original.at[idx, 'secondary score'] = secondary_score
                basis_matrix_original.at[idx, 'secondary cluster'] = secondary_cluster_id
            else:
                basis_matrix_original.at[idx, 'secondary score'] = secondary_score
                basis_matrix_original.at[idx, 'secondary cluster'] = 'NA'
        else:
            basis_matrix_original.at[idx, 'secondary score'] = secondary_score
            basis_matrix_original.at[idx, 'secondary cluster'] = 'NA'

        # Tertiary score and cluster
        tertiary_cluster_id = tertiary_cluster[idx]
        if len(sorted_scores) > 2 and tertiary_cluster_id != 'NA':
            tertiary_score = sorted_scores.iloc[2]
            # Check if tertiary cluster meets the thresholds
            if tertiary_score >= 0.70 * primary_score and tertiary_score >= 0.05 * max_scores_per_column[tertiary_cluster_id]:
                basis_matrix_original.at[idx, 'tertiary score'] = tertiary_score
                basis_matrix_original.at[idx, 'tertiary cluster'] = tertiary_cluster_id
            else:
                basis_matrix_original.at[idx, 'tertiary score'] = tertiary_score
                basis_matrix_original.at[idx, 'tertiary cluster'] = 'NA'
        else:
            basis_matrix_original.at[idx, 'tertiary score'] = tertiary_score
            basis_matrix_original.at[idx, 'tertiary cluster'] = 'NA'

    # Create clustered_samples dictionary
    clustered_samples = {i: [] for i in range(n_components)}

    for column in clustered_samples.keys():
        temp_df = basis_matrix_original[basis_matrix_original['primary cluster'] == column].copy()
        temp_df['score'] = temp_df['primary score']
        for cluster_type in ['secondary', 'tertiary']:
            temp_cluster_column = f'{cluster_type} cluster'
            temp_score_column = f'{cluster_type} score'
            temp_df_secondary = basis_matrix_original[basis_matrix_original[temp_cluster_column] == column].copy()
            temp_df_secondary['score'] = temp_df_secondary[temp_score_column]
            temp_df = pd.concat([temp_df, temp_df_secondary])
        temp_df = temp_df.sort_values(by='score', ascending=False)
        clustered_samples[column] = temp_df.index.tolist()

    ### write bait and prey matrices to .csv
    basis_matrix_original.to_csv(f'7013_cleaned_v2_NMF_{n_components}_top10-07-005_prey_matrix.csv')
    scores_df_original.to_csv(f'7013_cleaned_v2_NMF_{n_components}_top10-07-005_bait_matrix.csv')

    # Initialize g:Profiler
    gp = GProfiler(return_dataframe=True)

    # Create a new Excel writer for each k value
    with pd.ExcelWriter(f'7013_cleaned_v2_NMF_{n_components}_top10-07-005_clusters_enrichment.xlsx') as writer:
        for label, preys in clustered_samples.items():

            # Query g:Profiler for GO terms
            results = gp.profile(organism='gp__2aO3_xfUn_hEA', query=preys, sources=[])
            # print(results)

            # Write to a separate sheet in the Excel file
            results.to_excel(writer, sheet_name=f'Cluster_{label}', index=False)

        # Find the maximum length among the lists in clustered_samples
    max_length = max(len(indices) for indices in clustered_samples.values())

    # Pad each list in clustered_samples to match the maximum length
    for key in clustered_samples:
        clustered_samples[key].extend([None] * (max_length - len(clustered_samples[key])))

    # Convert clustered_samples to a DataFrame and write to a CSV file
    clustered_samples_df = pd.DataFrame.from_dict(clustered_samples, orient='index').transpose()
    clustered_samples_df.to_csv(f'7013_cleaned_v2_NMF_{n_components}_top10-07-005_clusters.csv', index=False)


In [ ]:
# Call the function with the desired number of components
# k = 19 is what was used for the final published results
nmf_cluster_analysis_filtering(19)

In [ ]:
# Call the function with the desired number of components
# for Dyakov et al. 2014 we tested k = 10 to 24 and conducted precision-recall analysis to determine that k = 19 should be used
# see Methods and associated R code in the repo for this manuscript
nmf_cluster_analysis_filtering(10)
nmf_cluster_analysis_filtering(11)
nmf_cluster_analysis_filtering(12)
nmf_cluster_analysis_filtering(13)
nmf_cluster_analysis_filtering(14)
nmf_cluster_analysis_filtering(15)
nmf_cluster_analysis_filtering(16)
nmf_cluster_analysis_filtering(17)
nmf_cluster_analysis_filtering(18)
nmf_cluster_analysis_filtering(19)
nmf_cluster_analysis_filtering(20)
nmf_cluster_analysis_filtering(21)
nmf_cluster_analysis_filtering(22)
nmf_cluster_analysis_filtering(23)
nmf_cluster_analysis_filtering(24)

# Transforming NMF prey matrix into prohits-viz compatible file (Fig. 3a, b & Extended Data Fig. 6)

This code is to transform data for creation of top N preys per rank/cluster, which can be made into a heatmap in prohits-viz, where clustering occurs.

In [ ]:
import pandas as pd

# Load the data from a CSV file
file_path = '7013_cleaned_v2_NMF_19_top10-07-005_prey_matrix-UpdatedHeaders.csv'
data = pd.read_csv(file_path)

# Specify the column to sort by and the number of top rows to select
sort_column_index = 15  # column in excel numbering (not python) corresponding to cluster to sort by, e.g. use 1 for PS (cluster 0) and 13 for NS (cluster 12)
top_n = 50  # Number of top entries based on the sort column

# Sort the data in descending order by the specified column and select the top N entries
sorted_data = data.sort_values(by=data.columns[sort_column_index], ascending=False).head(top_n)

# Prepare the data for melting (transforming)
output_columns = ['PreyGene'] + list(data.columns[1:20])  # Columns from 2 to 20 are cluster scores
output_data = sorted_data[output_columns]

# Melt the data into a long format
melted_data = output_data.melt(id_vars=['PreyGene'], var_name='Cluster', value_name='NMF Score')

# Replace column header numbers with 'Cluster x'
melted_data['Cluster'] = melted_data['Cluster']#.apply(lambda x: 'Cluster ' + x)

# Dynamically name the output file based on the selected column and top N
output_file_name = f"output_sorted_by_cluster_{sort_column_index-1}_top_{top_n}.txt"
output_file_path = output_file_name  # Adjust this path as necessary

# Save the transformed data to a tab-delimited txt file
melted_data.to_csv(output_file_path, sep='\t', index=False)

print(f"Output saved to {output_file_path}")


Output saved to output_sorted_by_cluster_14_top_50.txt


# Prey-prey Pearson correlations based on NMF scores (Fig. 2a)

code used to compute prey-prey Pearson correlations to be added here

##Filtering prey-prey correlations to produce input files for cytoscape (For Fig. 2a)



In [ ]:
import pandas as pd

# Load the data from the input CSV file
df = pd.read_csv('preys_NMF_correlations_all.csv')

# Apply the filter to retain rows where 'Correlation' is greater than or equal to 0.5 or other value
filtered_df = df[df['Correlation'] >= 0.6]

# Write the filtered data to a new CSV file
filtered_df.to_csv('preys_NMF_correlations_06.csv', index=False)
